In [1]:
import matplotlib.pyplot as plt
from graphics import *
from operator import *
from random import *
import numpy as np
import timeit
from math import *

a = 3
s = 3
r = 3
rr = 20
ro = 70
ra = 300
noise = 0.01
prop = 0.4
weight = 3
bias = np.array([0.0,-1.0])
dev_bias = 3
dTheta = 120
sight_theta = 180


In [24]:
def initialize_agents(speed, N, width, height):
    """
    Initializes our agent set with randomly directed speeds, draws the window and the agents
    """
    seed()

    agents = [np.array([0.0, 0.0]) for i in range(N)]
    #agents = [Point(uniform(0, width), uniform(0, height)) for i in range(N)]
    
    speeds = N*[0,0]
    #speeds = [np.array([0.0, 0.0]) for i in range(N)]
    
    for i in range(N):
        theta = uniform(0, 2 * np.pi)
        #speeds[i][0] = speed * np.cos(theta)
        #speeds[i][1] = speed * np.sin(theta)
        speeds[i]= speed * np.array([np.cos(theta),np.sin(theta)])
        
        theta = uniform(0, 2 * np.pi)
        radius = uniform(0,height/2)
        agents[i] = Point(width/2 + radius*np.cos(theta), height/2 + radius*np.sin(theta))

    return agents, speeds


def initialize_window(agents, width, height):
    win = GraphWin("Swarm", width, height) # size of box
    for agent in agents:
        agent.draw(win)
    win.getMouse()
    return win


def couple_speeds(agents, speeds, a, s, N):
    """
    Simplest model: for each agent, it will give its nearest neighbour a fraction of its speed and re-normalize it
    
    """
    nearest_neighbours = [nearest_neighbour(agent, agents, N) for agent in agents]
    for i in range(N):
        weightedSpeed = a * speeds[nearest_neighbours[i]]
        speeds[i] = speeds[i] + weightedSpeed
        speeds[i] = s * normalized(speeds[i])


def angle(vec1, vec2):  # (Not so) stupid way of doing an angle
    return np.arccos(np.dot(normalized(vec1),normalized(vec2))) # radians
            
def get_distances(agent, agents, N):
    """
    Given one angent and the set of all agents, 
    computes the distances from the first to all of the others
    """
    dists = N * [0.0]
    for i in range(N):
        a, dists[i] = relative_pos(agent, agents[i])
        if dists[i] == 0:
            dists[i] = 0.1
    
    return dists

def relative_pos(agent1, agent2):
    dx = agent2.getX() - agent1.getX()
    dy = agent2.getY() - agent1.getY()

    return np.array([dx, dy]), np.linalg.norm([dx, dy])

def nearest_neighbour(agent, agents, N):
    """
    Returns the index for the agent with smallest Eucledian distance to agent in question
    """
    distances = get_distances(agent, agents, N)
    j = next(i for i in range(N) if agents[i] is agent)
    distances[j] = distances[j-1] + 1

    return distances.index(min(distances))


def softened_angle(speed, newspeed, s, maxTheta):
    theta = angle(speed, newspeed)
    if maxTheta**2 > theta**2:
        return s*newspeed
    else:
        return np.dot(rot_matrix(maxTheta), speed)    

def in_sight_range(rel_pos, speed1, angle_range):
    return 360*angle(speed1, rel_pos)/np.pi < angle_range
    

def noisy_vector(noise):
    return noise * np.array([2 * random() - 1, 2 * random() - 1])
    
def biaser(speeds, N, s, i, prop, bias, dev_bias, weight):
    #bias = np.array([0.0,1.0])
    #Ns has to be integer 
    Ns = int(N * prop)
    gbias = np.random.normal(bias, dev_bias, )
    for i in range(Ns):
        tot_dir = normalized(normalized(speeds[i]) + weight * gbias)
        
        if np.linalg.norm(tot_dir) != 0:
            speeds[i] = s*tot_dir
    #bias = np.dot(tot_dir,np.array([[np.cos(rot_bias*i), 0],[0, np.sin(rot_bias*i)]]))
    #return bias

def quality (agents, speeds, N, bias, window, old_cm):
#ACCURACY (DIRECTIONS DISTRIBUTION)
    dev = 0.0
    for i in range(N):
        dev += angle(bias, speeds[i])
    dev_avg = dev/(N*2*(np.pi))
    #print (dev_avg)
    
#ELONGATION: SHAPE OF SWARM
    #can be smarter if we make agents become poses before, more globally in  the code
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    #Center of Mass
    cm = np.mean(poses,axis=0)
    #Standard Deviation
    std = np.std(poses,axis=0)
    #print (cm, std)

    #Elongation >1 means 
    elong = std[1] / std[0]
    #Drawing it
    #old_cm = Point(cm[0],cm[1])
    #old_cm.draw(window)
    #old_cm.setFill("red")

#GROUP DIRECTION
    #Vector
    group_dir = np.array([cm[0] - old_cm[0], cm[1] - old_cm[1]])                
    #print (group_dir)
    # Norm
    group_dir = np.linalg.norm(group_dir)
    #print (group_dir)
    return cm
    


def rigid_boundary(x_bound, y_bound, agents, speeds, N):
    for i in range(N):
        [dx, dy] = [0, 0]
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            speeds[i][0] = -speeds[i][0]
            dx = x_bound - x         
        
        elif x < 0:
            speeds[i][0] = -speeds[i][0]
            dx = -x
        
        if y > y_bound:
            speeds[i][1] = -speeds[i][1]
            dy = y_bound - y
        
        elif y < 0:
            speeds[i][1] = -speeds[i][1]
            dy = -y

        agents[i].move(dx, dy)
        
        
def periodic_boundary(x_bound, y_bound, agents, speeds, N):  #Changed from rigid boundaries do periodic boundary condition
    [dx, dy] = [0, 0]
    for i in range(N):
        [x, y] = [agents[i].getX(), agents[i].getY()]
        if x > x_bound:
            dx = -x_bound         
        
        elif x < 0:
            dx = x_bound
        
        if y > y_bound:
            dy = -y_bound
        
        elif y < 0:
            dy = y_bound

        agents[i].move(dx, dy)

def next_step(agents, speeds, dt, N):
    dxvec = [dt * speeds[i][0] for i in range(N)]
    dyvec = [dt * speeds[i][1] for i in range(N)]
    for i in range(N):
        agents[i].move(dxvec[i], dyvec[i])
        

def normalized(vector):
    if vector[0] == vector[1] == 0:
        return vector
    return vector / np.linalg.norm(vector)


def rot_matrix(theta, unit = "None"):
    if unit in ["None", "deg"]:
        c, s = np.cos(pi * theta / 180), np.sin(pi * theta / 180)
    elif unit == "rad":
        c, s = np.cos(theta), np.sin(theta)
    return np.array([[c, -s],[s, c]])

def warn_me_args(N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dTheta):
    if dt*s < rr:
        print("Warning - step length bigger the repulsion radius.")
        

        
def virtualizer (current, agents, h, w, N):
    lower_limit = current.getY() - h / 2
    upper_limit = current.getY() + h / 2
    left_limit = current.getX() - w / 2
    right_limit = current.getX() + w / 2
    
    vagents=[np.array([agent.getX(),agent.getY()]) for agent in agents]
    virtuals = N * [0.0, 0.0]
    vvirtuals = N * [Point(0.0, 0.0)]

    for j in range(N):
        #make more compact!!!
        virtuals[j]=vagents[j]
        
        #newcopy = agents[i].clone()
        candidates = [vagents[j],
                      vagents[j]+[w,0],vagents[j]+[-w,0],vagents[j]+[w,h],vagents[j]+[w,-h],
                      vagents[j]+[-w,h],vagents[j]+[-w,-h],vagents[j]+[0,h],vagents[j]+[0,-h]]
        #print candidates
        
        #virtuals[j] = next((cand for cand in candidates if lower_limit < cand[1] < upper_limit and left_limit < cand[0] < right_limit),False
        for i in range(9):
            if lower_limit < candidates[i][1] < upper_limit and left_limit < candidates[i][0] < right_limit:
                virtuals[j]=candidates[i]
            
        vvirtuals[j] = Point(virtuals[j][0],virtuals[j][1])
        
    return vvirtuals


def get_cm_std(agents, N):
    poses = [np.array([a.getX(), a.getY()]) for a in agents]
    return np.mean(poses,axis = 0), np.std(poses,axis = 0)

def mill_observables (N, agents, speeds):
    cm, std = get_cm_std(agents, N)
    mean_R = np.linalg.norm(std)
    point_cm = Point(cm[0],cm[1])
    norm_R = N*[0.0]
    vector_R = N*[0,0]
    versor_R = N*[0,0]
    versor_T = N*[0,0]
    
    proj_radial = N*[0]
    radial = N*[0,0]
    
    proj_tangencial= N*[0]
    norm_tangencial= N*[0]
    tangencial= N*[0,0]
    square = 0
    angles = N*[0]
    angular= N*[0]
    
    for i in range(N):
        vector_R[i],norm_R[i] = relative_pos(point_cm, agents[i])
        
        versor_R[i] = normalized(vector_R[i])
        proj_radial[i] = np.dot(speeds[i], versor_R[i])
        radial[i] = proj_radial[i]*versor_R[i]
        
        angular[i] = np.cross([vector_R[i][0],vector_R[i][1],0],[speeds[i][0],speeds[i][1],0])[2]
       
        versor_T[i] = np.dot(versor_R[i],rot_matrix(90,'deg'))
        norm_tangencial[i]= norm_R[i]*angular[i]
        proj_tangencial[i] = np.dot(speeds[i], versor_T[i])
        tangencial[i] = proj_tangencial[i]*versor_T[i]
        
        angles[i] = (np.pi - angle(vector_R[i],speeds[i]))*360/(2*np.pi)
        
        square += (np.linalg.norm(speeds[i]))**2
        
    square = square / N
    
    print np.linalg.norm(speeds[0]), square # square is converging to alpha / beta !!! amazing!
    
    print proj_radial[0], radial[0]
    print angular[0], norm_R[0], norm_tangencial[0] # norm_tangencial should be the same as proj_tangencial
    #angular momentum vs velocityproblem?
    print proj_tangencial[0], tangencial[0]
    
    print angles[0]
    
    
    min_R = min(norm_R)
    max_R = max(norm_R)
    
    #return mean_R, min_R, max_R, 
    #return angles



In [27]:

##MILL MODEL
def mill (dt, agents, speeds, N, width, height, cr, ca, lr, la, alpha, beta, mass):
    clr = cr / lr
    cla = ca / la
    for i in range(N):
        u_dir = np.array([0.0, 0.0])
        propulsion = np.array([0.0, 0.0])
        friction = np.array([0.0, 0.0])
        grad_U = np.array([0.0, 0.0])
        
        #virtuals = virtualizer(agents[i], agents, height, width, N)
        
        for j in range(N):   #Duality interactions, by the Morse potential  
            if i == j:
                #Eliminate the i-i interaction
                continue

            rel_pos, distance = relative_pos(agents[i], agents[j])
            u_dir = normalized(rel_pos)
            grad_U = grad_U + u_dir * (clr*np.exp(-distance / lr) - cla *np.exp(- distance / la))
            
        
        propulsion = alpha * speeds[i] # self-propulsion propto alpha
        norm = (np.linalg.norm(speeds[i]))
        friction =  beta *((norm)**2) * speeds[i] #friction force propto beta
     
        d_speed = (propulsion - friction - grad_U)/mass
        speeds[i]= speeds[i]+dt*d_speed
        #print u_dir
    
   # print speeds[N-1], propulsion, friction, grad_U
    return

def run(N_steps, N, s, dt, width, height):#N_steps, a, dt, N, width, height, s, rr, ro, ra, noise, prop, weight, bias, dev_bias, dTheta, sight_theta):
    """
    Simulates motion of swarm. Recieves following parameters:

    N_steps  - number of steps to perform
    a -  coupling between neighbouring points
    dt - time step to be used 
    N - number of points to be used
    width & heigth of window
    s - module of speed throughout agents

    """
    
    agents, speeds = initialize_agents(s, N, width, height)
    window = initialize_window(agents, width, height)
    old_cm=np.array([0.0,0.0])
    bias = np.array([0.0,-1.0])
    for i in range(N_steps):
        
        #print 'step', i
        
        next_step(agents, speeds, dt, N)
        start = timeit.default_timer()
        ## MODEL
        #couple_speeds(agents, speeds, a, s, N)
        #vicsek(agents, speeds, N, s, 0.1, r = 500)
        #couzin(agents, speeds, N, width, height, s, noise, dTheta, rr, ro, ra, sight_theta, 1, 200, 1.5, 1) #(...)last 4 parameters: model2, roa, orient, atract
        
        mill (dt, agents, speeds, N, width, height, cr=120, ca=250, lr=120, la=250.0, alpha=2.0, beta=0.01, mass=1) 
        # Region II, really nice ring
        #mill (dt, agents, speeds, N, width, height, cr=110, ca=100, lr=20, la=100.0, alpha=4.0, beta=0.06, mass=3) 
        # Region VII,(l<1/sqrt(C)) alpha ~ beta, best vortex in this region, decreasing in size with increasing N
        
        mill_observables (N, agents, speeds)
        print i
        #print angles
        
        
        #interaction(agents, speeds)
        #biaser(speeds, N, s, i, prop, bias, dev_bias, weight)
        
        ## BOUNDARY CONDITIONS
        rigid_boundary(width, height, agents, speeds, N)
        #periodic_boundary(width, height, agents, speeds, N)

        #time.sleep(0.02)
        stop = timeit.default_timer()
        #print stop - start
    window.close()
    return

In [28]:

run(100000, 50, 5, 0.3, 900, 700)

7.79060279083 55.1175857769
-3.38260869855 [-0.07712614  3.38172932]
224.030713369 31.9225848688 7151.63946075
-7.01793774816 [ 7.01611328  0.1600145 ]
64.2661803734
0
11.1762806086 113.241188825
-4.36130592746 [-0.43929962  4.33912494]
320.425035957 31.1388477565 9977.66641202
-10.2902020801 [ 10.23786756   1.0364973 ]
67.0312668273
1
13.7662765524 166.463230717
-4.19359712336 [-0.90627819  4.09449834]
396.898152721 30.2698787286 12014.0589505
-13.1119835755 [ 12.80213464   2.83363052]
72.2641983629
2
14.2006866485 184.016579256
-2.63227840111 [-0.94631975  2.45629162]
414.572335186 29.7086695945 12316.3925291
-13.954591062 [ 13.02162606   5.01675855]
79.3177174622
3
14.0617001274 196.16014395
-0.817209631048 [-0.40988025  0.7069864 ]
418.074385439 29.7817612101 12450.9915152
-14.0379335691 [ 12.14453144   7.04087602]
86.6683217846
4
14.0249747033 201.558776995
0.888863356279 [ 0.55822887 -0.69170701]
426.999631573 30.5069913042 13026.4740473
-13.9967795354 [ 10.89219226   8.79033475]

14.2276804084 214.273997354
-3.4736169957 [ 2.98133871 -1.78259208]
345.250822851 25.0233745448 8639.34065211
-13.797132846 [ -7.08041784 -11.84181396]
75.8686886293
52
14.1606357012 213.180644437
-1.97739246182 [ 1.84951142 -0.69956305]
336.265367138 23.9814505507 8064.13127394
-14.0218943982 [ -4.96067393 -13.11507669]
81.9729795773
53
14.0881091031 212.061557903
-0.335411691235 [ 0.33029045 -0.05838852]
330.106597974 23.4382195732 7737.11092588
-14.0841157727 [ -2.45176511 -13.86907225]
88.6357650876
54
14.0105445974 210.903022398
1.3386210246 [-1.33847428 -0.01982073]
326.744315392 23.4284944982 7655.12739549
-13.9464495005 [  0.20650267 -13.94492059]
95.4826210847
55
13.9290149775 209.687827379
2.91581280633 [-2.85735631 -0.58092966]
326.192261095 23.9487891129 7811.90967122
-13.6204072598 [  2.71365106 -13.347344  ]
102.083310602
56
13.8450957669 208.408473852
4.29123431005 [-3.99128389 -1.57618046]
328.495750681 24.9554600729 8197.76259023
-13.1632816915 [  4.83490434 -12.243189

14.2653095696 200.144854962
-3.40063397005 [-0.20334095 -3.39454914]
252.831225031 18.2496234561 4614.07465477
-13.8540515994 [-13.82926223   0.82840317]
76.2087663161
104
14.1709628254 200.537139982
-1.13634741342 [ 0.17054675 -1.12347641]
250.355583807 17.7238770669 4437.27159041
-14.125328384 [-13.96533584  -2.11997569]
85.4006026035
105
14.0714251101 200.770217967
1.23315831147 [-0.44455638  1.15023869]
249.713610055 17.8146895455 4448.57043842
-14.0172866565 [-13.07474094  -5.05326375]
95.0276015624
106
13.9663205161 200.781030981
3.44812758625 [-1.89288537  2.88211187]
250.897293174 18.5383275747 4651.21620846
-13.5339766849 [-11.31235255  -7.42961673]
104.293481229
107
13.8555349547 200.577815059
5.30684085732 [-3.71791284  3.78677752]
253.986580696 19.8443193028 5040.19080594
-12.7989565588 [-9.13289136 -8.96680458]
112.520399172
108
13.7389448788 200.192263899
6.73184332643 [-5.46874171  3.92563098]
259.147931013 21.6377080902 5607.36728344
-11.9766811685 [-6.98412432 -9.72948

12.6637828111 197.458074356
8.55777806283 [ 6.40152797  5.67943703]
488.949867456 52.3800546573 25611.2207821
-9.33465745034 [-6.19501918  6.98266189]
132.513811371
156
12.7038788977 197.035909072
7.91129177915 [ 5.6626252   5.52478176]
549.211752192 55.2536975856 30345.9800661
-9.93981898391 [-6.94138607  7.11457383]
128.516952003
157
12.7887181818 196.487482498
7.11831620285 [ 4.8530677   5.20750991]
615.77397343 57.9576972392 35688.8415199
-10.6245417393 [-7.77254126  7.24351361]
123.821613408
158
12.9219337857 195.878029379
6.15729086429 [ 3.97648961  4.70103831]
686.626553443 60.4390730984 41499.0724549
-11.3606400337 [-8.67375039  7.33690644]
118.457006502
159
13.1035511256 195.272814117
5.01183427766 [ 3.04559141  3.9803085 ]
758.332883075 62.6348094887 47498.0356604
-12.1072114574 [-9.61532924  7.35731016]
112.487327827
160
13.3283965145 194.741790568
3.67731321776 [ 2.08565972  3.0286393 ]
825.988054985 64.4745550819 53255.2123481
-12.8110702577 [-10.55121187   7.2660477 ]
106

13.2462315846 193.038677567
4.83097387895 [ 3.36326672 -3.46795986]
661.1553031 53.6048553656 35441.1343968
-12.3338697323 [ 8.85398394  8.58669383]
111.389461498
208
13.3720483771 192.068354149
3.99494934218 [ 3.00247356 -2.63529368]
695.608034926 54.5089686405 37916.8765619
-12.7613501461 [ 8.4181056   9.59101432]
107.382764687
209
13.513778614 191.207008142
3.0862714466 [ 2.47896948 -1.83841828]
727.369888113 55.2853891165 40212.9272959
-13.156638666 [  7.83709581  10.56773724]
103.201691181
210
13.6670001033 190.495989572
2.11461053599 [ 1.79819157 -1.11269259]
755.071598581 55.9212100001 42224.5174294
-13.5024188243 [  7.10487399  11.48198936]
98.9007793811
211
13.8260542482 189.971168685
1.09454054777 [ 0.97653598 -0.49436474]
777.412397806 56.4050999706 43850.0240166
-13.7826614652 [  6.22513429  12.29672558]
94.5405755386
212
13.9848722097 189.642337548
0.0447854549276 [ 0.04155808 -0.01669319]
793.325798868 56.7277165627 45003.5610601
-13.9848004986 [  5.21265032  12.97701513]

...

13.9396559045 201.87503506
1.21258286027 [ 0.05943752  1.21112525]
496.943482419 35.7852724565 17783.2579138
-13.8868156732 [-13.87012275   0.68069405]
94.9903527062
514
13.9226083801 201.646145016
1.32064100232 [-0.09874358  1.31694433]
509.961454803 36.7942028609 18763.6252193
-13.8598315808 [-13.82103581  -1.03629179]
95.4430237803
515
13.9161750478 201.34725847
1.30962382814 [-0.25117211  1.28531208]
524.021018271 37.8233956799 19820.2543187
-13.8544149421 [-13.59722273  -2.65713145]
95.399983733
516
13.9219799903 200.981851713
1.18909019966 [-0.35833323  1.13381339]
538.475826771 38.8199623895 20903.6113429
-13.8711063491 [-13.2262852   -4.18006833]
94.899659247
517
13.9391398184 200.56126202
0.971738990546 [-0.3918557   0.88922763]
552.548911986 39.7367771025 21956.5129538
-13.9052271542 [-12.72451998  -5.60731072]
93.9975019731
518
13.9676332475 200.105013024
0.672624994029 [-0.33466244  0.58345988]
565.497575856 40.533310282 22921.4887059
-13.9514283912 [-12.1019868   -6.941489

14.1410880698 197.827230673
-0.646682888779 [-0.43803692 -0.47573355]
418.94255251 29.6569335241 12424.5514302
-14.1262936766 [-10.39203601   9.56858195]
87.3789055861
562
14.0654087831 197.924852744
0.272923854137 [ 0.15042105  0.22772997]
420.766573084 29.9206239706 12589.5984126
-14.0627606395 [-11.73408617   7.75064246]
91.1118316328
563
13.9928314335 198.064813616
1.10806481342 [ 0.45899039  1.00853134]
426.244256084 30.5575759229 13024.991217
-13.9488897012 [-12.69591111   5.77800702]
94.5418954503
564
13.9256876507 198.246417842
1.81019918476 [ 0.49506239  1.74118762]
435.362335266 31.5307846138 13727.3160223
-13.8075325622 [-13.28113774   3.77615358]
97.4690105009
565
13.8664415322 198.467786947
2.3487626881 [ 0.31674409  2.32730732]
448.0454486 32.785240687 14689.2778711
-13.6660716594 [-13.54123545   1.84294796]
99.7520446804
566
13.8178360304 198.72518901
2.71100682484 [ 0.00698023  2.71099784]
464.138763096 34.2555991828 15899.3514338
-13.549281699 [-13.54923679   0.0348863

14.4127045408 198.599284523
-3.56349327605 [-3.24787407 -1.46621907]
505.590418515 36.2035228664 18304.1542777
-13.9652270963 [ -5.74607014  12.72832455]
75.685355792
614
14.3961239077 198.418636059
-3.42403783654 [-2.93200137 -1.76844652]
493.225352618 35.2732087264 17397.640812
-13.9830021262 [ -7.22193871  11.97363561]
76.2406484767
615
14.370678302 198.267320007
-3.17894976376 [-2.49828337 -1.96578275]
482.251001942 34.4104693606 16594.4833265
-14.0146592273 [ -8.66631353  11.01388592]
77.219838822
616
14.3377746526 198.143537786
-2.83638857736 [-1.98499247 -2.02605653]
473.131602182 33.6642600506 15927.625294
-14.0544185873 [-10.03919095   9.8357169 ]
78.5901256781
617
14.2984831467 198.051034057
-2.41169254222 [-1.44377155 -1.93178276]
466.203041219 33.0789936028 15421.5274181
-14.0936283255 [-11.28909582   8.43721962]
80.2896335214
618
14.2554726783 198.001797285
-1.92814210163 [-0.93318062 -1.68727766]
461.744128642 32.6910669058 15094.9082028
-14.1244741254 [-12.36003798   6.8

14.212195779 200.00148698
-1.26881494519 [-1.21129165  0.37770876]
578.694376646 40.8813982753 23657.8352914
-14.1554448003 [  4.2138813   13.51369017]
84.878014569
662
14.2632716589 200.208952205
-1.78400109585 [-1.74388556  0.37619552]
579.595605316 40.9571629785 23738.5916685
-14.151263495 [  2.98410241  13.8330543 ]
82.8148223786
663
14.311763046 200.390321762
-2.28654281846 [-2.26934675  0.27989893]
576.573408436 40.8109033382 23530.4816391
-14.1279256589 [  1.7294193  14.0216758]
80.8066464374
664
14.3555730421 200.546776197
-2.75731591977 [-2.75587569  0.08910826]
569.811593509 40.4457831285 23046.4761352
-14.0882818784 [  0.45529141  14.08092312]
78.9262214936
665
14.3932370144 200.682055003
-3.17804899668 [-3.17243631 -0.1887943 ]
559.743752594 39.8734855908 22318.9344536
-14.0379940279 [ -0.83393718  14.01320181]
77.2438839532
666
14.4229596547 200.788195593
-3.53179934634 [-3.4903727  -0.53935614]
546.96402194 39.1139697025 21393.9341825
-13.9838534953 [ -2.13553391  13.8198

KeyboardInterrupt: 

KeyboardInterrupt: 